In [ ]:
import glob
import asyncio

import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import aiohttp

from shapely import Point
from tqdm.asyncio import tqdm

from potentiel_solaire.constants import DATA_FOLDER, RESULTS_FOLDER, CRS_FOR_BUFFERS, DATABASE_FOLDER
from potentiel_solaire.database.queries import get_connection

### Constants

In [ ]:
OUTPUT_INITIAL_POSITIONS_PATH = DATABASE_FOLDER / "correction_adresses" / "positions_initiales_etablissements.geojson"
OUTPUT_CORRECTIONS_PATH = DATABASE_FOLDER / "correction_adresses" / "positions_corrigees_etablissements.geojson"

### Fonctions utiles

In [ ]:
def cree_adresse(df: pd.DataFrame, cols: list[str] = None) -> pd.Series:
    """
    Construit une adresse complète à partir des colonnes spécifiées d'un DataFrame.

    Args:
        df (pd.DataFrame): Le DataFrame contenant les colonnes d'adresse.
        cols (list[str], optional): Liste des noms de colonnes à utiliser pour l'adresse.
            Par défaut ["adresse_1", "code_postal", "nom_commune"].

    Returns:
        pd.Series: Série contenant les adresses concaténées.
    """
    if cols is None:
        cols = ["adresse_1", "code_postal", "nom_commune"]
    return df[cols].fillna(" ").apply(lambda x: ' '.join(x.values), axis=1)


async def geocode_adresse(
    adresse: str, 
    initial_attrs: dict,
    session: aiohttp.ClientSession, 
    pbar: tqdm
) -> dict:
    """
    Géocode une adresse en utilisant l'API IGN (data.geopf.fr).

    Args:
        adresse (str): L'adresse à géocoder, typiquement obtenue via cree_adresse().
        initial_attrs (dict): Dictionnaire contenant les attributs initiaux de l'établissement,
        session (aiohttp.ClientSession): La session HTTP asynchrone à utiliser pour la requête.
        pbar (tqdm): La barre de progression à mettre à jour après chaque géocodage.

    Returns:
        dict : Dictionnaire contenant les attributs initiaux et la géométrie (Point) si trouvée.
    """
    async with session.get(url="https://data.geopf.fr/geocodage/search",
                            params={"q": adresse}) as response:
        
        results = initial_attrs
        
        if response.status == 200:
            js = await response.json()
            
            features = js.get('features', [])
            if not features:
                return results
            
            coordinates = features[0].get('geometry', {}).get('coordinates', None)
            if not coordinates:
                return results
            
            results["geometry"] = Point(coordinates)
            pbar.update(1)

            return results

        if response.status == 429:
            await asyncio.sleep(0.04)
            return await geocode_adresse(
                adresse=adresse,
                initial_attrs=initial_attrs,
                session=session, 
                pbar=pbar
            )

        if response.status in [504, 529]:
            await asyncio.sleep(5)
            return await geocode_adresse(
                adresse=adresse,
                initial_attrs=initial_attrs,
                session=session, 
                pbar=pbar
            )

        print(f"Erreur {response.status} pour l'adresse {adresse}")
        return results


def calcul_dist(gdf: gpd.GeoDataFrame, col1: str, col2: str, crs: str = CRS_FOR_BUFFERS) -> pd.Series:
    """
    Calcule la distance entre deux colonnes de géométrie dans un DataFrame.

    Args:
        gdf (gpd.GeoDataFrame): Le GeoDataFrame contenant les colonnes de géométrie.
        col1 (str): Nom de la première colonne.
        col2 (str): Nom de la seconde colonne.
        crs (str, optional): Système de référence de coordonnées cible (projetées)
                             Par défaut EPSG:6933 (utilisée par le projet).

    Returns:
        pd.Series: Série contenant les distances calculées.
    """
    return gpd.GeoSeries(gdf[col1], crs=gdf.crs).to_crs(crs)\
        .distance(gpd.GeoSeries(gdf[col2], crs=gdf.crs).to_crs(crs))

### Recuperation des etablissements au format des .geojson

In [ ]:
output_path = f"{DATA_FOLDER}/etablissements.geojson"

with get_connection() as conn:
    export_query = f"""
        COPY (
        SELECT 
            identifiant_de_l_etablissement,
            nom_etablissement,
            code_commune,
            code_departement,
            code_region,
            adresse_1,
            code_postal,
            nom_commune,
            reussite_rattachement,
            identifiant_topo_zone_rattachee,
            geom,
        FROM
            etablissements
        ) TO '{output_path}' WITH (FORMAT GDAL, DRIVER 'GeoJSON', LAYER_NAME 'Etablissements')
    """

    conn.query(export_query)

# Lecture du fichier GeoJSON
etablissements = gpd.read_file(output_path)

# Ajout d'une colonne pour l'adresse complète
etablissements["adresse_complete"] = cree_adresse(etablissements)

etablissements.head()

### Recuperation des zones d'education

Il faut executer en amont les commandes suivantes : 
```bash
poetry run extract-data -a
poetry run algorithme attach-buildings-to-schools -a
```

In [ ]:
files = glob.glob(f"{RESULTS_FOLDER}/D*.gpkg")
gdfs = [gpd.read_file(f, layer="educational_zones") for f in files]
educational_zones = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True), crs=gdfs[0].crs)
educational_zones["zone_geometry"] = educational_zones.geometry

educational_zones.head()

### Calcul de la distance entre la position de l'etablissement et sa zone d'education

In [ ]:
etablissements_with_zones = etablissements[etablissements["reussite_rattachement"]].merge(
    educational_zones[["identifiant_de_l_etablissement", "cleabs_grande_zone", "zone_geometry"]],
    how="left",
    left_on=["identifiant_de_l_etablissement", "identifiant_topo_zone_rattachee"],
    right_on=["identifiant_de_l_etablissement", "cleabs_grande_zone"],
)

etablissements_with_zones["distance_zone_etablissement_m"] = calcul_dist(
    etablissements_with_zones, "geometry", "zone_geometry"
)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
bins=np.logspace(-3, 5, 81)
plt.hist(etablissements_with_zones["distance_zone_etablissement_m"], bins=bins)
plt.xlabel("Distance entre la position de l'établissement et sa zone d'éducation (mètres)")
plt.ylabel("Nombre d'établissements concernés")
plt.title("Distribution de la distance (échelle log/log)")
ax.loglog()
ax.grid()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
plt.hist(etablissements_with_zones["distance_zone_etablissement_m"], bins=bins, cumulative=True, density=True)
plt.xlabel("Distance entre la position de l'établissement et sa zone d'éducation (mètres)")
plt.ylabel("Nombre d'établissements concernés")
plt.title("Distribution cumulée de la distance")
ax.loglog()
ax.grid()
ax.set_xscale('log')

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
plt.hist(etablissements_with_zones["distance_zone_etablissement_m"], bins=bins, cumulative=True, density=True)
plt.xlabel("Distance entre la position de l'établissement et sa zone d'éducation (mètres)")
plt.ylabel("Nombre d'établissements concernés")
plt.title("Distribution cumulée de la distance")
ax.grid()
ax.set_xscale('log')
ax.set_ylim(0.8, 1)
ax.set_xlim(100, 1e5)

In [ ]:
quantiles = [0.25, 0.5, 0.75, 0.85, 0.9, 0.95, 0.96, 0.97, 0.98, 0.99, 1.0]
etablissements_with_zones["distance_zone_etablissement_m"].quantile(quantiles)

### Selection des etablissements qu'on souhaite corriger

In [ ]:
### Choix du seuil de correction
seuil_correction_m = 268 # Correspond au 95ème percentile

### Selection des etablissements qu'on souhaite corriger
etablissements_to_correct = etablissements_with_zones[
    etablissements_with_zones["distance_zone_etablissement_m"] >= seuil_correction_m
].copy()
print(f"Nombre d'établissements à corriger : {len(etablissements_to_correct)}")


### Sauvegarde de la position initiale des etablissements à corriger

In [ ]:
etablissements_to_correct[["identifiant_de_l_etablissement", "geometry"]].to_file(OUTPUT_INITIAL_POSITIONS_PATH, driver="GeoJSON")

### Correction de la position

In [ ]:
async def correct_adresses(etab: gpd.GeoDataFrame, seuil_correction_m: int, session: aiohttp.ClientSession, pbar: tqdm) -> list:
    # Geocodage des adresses
    tasks = []
    for _, row in etab.iterrows():
        adresse = row["adresse_complete"]
        initial_attrs = {
            column: row[column] for column in etab.columns
        }
        
        tasks.append(
            geocode_adresse(
                adresse=adresse, 
                initial_attrs=initial_attrs,
                session=session, 
                pbar=pbar
            )
        )

    # Récupération des résultats
    results = await asyncio.gather(*tasks, return_exceptions=True)
    gdf = gpd.GeoDataFrame(results, crs=etab.crs)

    # Calcul de la distance entre la position corrigée et la zone
    gdf["distance_zone_adresse_geocodee_m"] = calcul_dist(
        gdf, "geometry", "zone_geometry"
    )

    # Si la nouvelle geometry est a une distance plus faible que le seuil, on la garde, sinon on prend le centre de la zone
    gdf["geometry"] = np.where(
        gdf["distance_zone_adresse_geocodee_m"] < seuil_correction_m,
        gdf["geometry"],
        gdf["zone_geometry"].apply(lambda geom: geom.centroid)
    )

    # Sauvegarde des résultats
    gdf[["identifiant_de_l_etablissement", "geometry"]].to_file(OUTPUT_CORRECTIONS_PATH, driver="GeoJSON", mode='a', index=False)

In [ ]:
with tqdm(total=len(etablissements_to_correct), desc="Processing adresses") as pbar:
    async with aiohttp.ClientSession() as session:
        etab_chunks = [
            etablissements_to_correct[i : i + 100]
            for i in range(0, len(etablissements_to_correct), 100)
        ]
        
        for chunk in etab_chunks:
            await correct_adresses(etab=chunk, seuil_correction_m=seuil_correction_m, session=session, pbar=pbar)